In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import tree
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import pickle 
from numpy import loadtxt
from keras.models import load_model
pd.set_option('display.max_columns', None)

In [2]:
big_df = pd.read_csv("data/big_df.csv")
big_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,Orthodox,Orthodox,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148.0,606.0,382.0,1659.0,845.0,25.0,51.0,47.

In [3]:
past_df = big_df[big_df["winner"] != "tbd"]
winners = past_df["winner"]
winners

1        red
2       blue
3        red
4       blue
5        red
        ... 
5803     red
5804     red
5805     red
5806     red
5807     red
Name: winner, Length: 5793, dtype: object

In [4]:
future_df = big_df[big_df["winner"] == "tbd"]
future_df

,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,Orthodox,Orthodox,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148.0,606.0,382.0,1659.0,845.0,25.0,51.0,47.

In [5]:
number = LabelEncoder()
# data['red_name'] = number.fit_transform(data['red_name'].astype('str'))
# data['blue_name'] = number.fit_transform(data['blue_name'].astype('str'))
future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))
future_df

<ipython-input-5-be9019f59664>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df['red_stance'] = number.fit_transform(future_df['red_stance'].astype('str'))
<ipython-input-5-be9019f59664>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df['blue_stance'] = number.fit_transform(future_df['blue_stance'].astype('str'))


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,0,0,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148.0,606.0,382.0,1659.0,845.0,25.0,51.0,47.0,69.0,58.0,10

In [6]:
data = future_df.drop(columns=["lev1", "winner", \
                                   "red_name", "blue_name"
                                  ])
target = future_df["winner"]
feature_names = data.columns

In [7]:

X_scaler = MinMaxScaler().fit(data)
X_scaled = X_scaler.transform(data)

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(winners)
encoded_y = label_encoder.transform(winners)

# Step 2: Convert encoded labels to one-hot-encoding
y_categorical = to_categorical(encoded_y)


In [8]:
loaded_model = load_model("saved_models/neural_up_model.h5")


In [9]:

# pred = loaded_model.predict(data)
encoded_predictions = loaded_model.predict_classes(X_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)
prediction_labels


/home/audricperger/anaconda3/envs/pyenv/lib/python3.8/site-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array(['blue', 'red', 'blue', 'blue', 'blue', 'blue', 'red', 'red',
       'blue', 'blue', 'red', 'blue', 'red', 'blue', 'blue'], dtype=object)

In [10]:
future_df["prediction"] = prediction_labels
future_df

<ipython-input-10-d0d58f6b7f7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["prediction"] = prediction_labels


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,prediction
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,0,0,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148.0,606.0,382.0,1659.0,845.0,25.0,51.0,47.0,6

In [11]:
pcts = loaded_model.predict_proba(data)
favs = []
dogs = []
for pct in pcts:
    print(pct)
    fav = max(pct)
    dog = min(pct)
    favs.append(fav)
    dogs.append(dog)

/home/audricperger/anaconda3/envs/pyenv/lib/python3.8/site-packages/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]
[0. 1.]
[1. 0.]
[0. 1.]


In [12]:
future_df["fav_pct"] = favs
future_df["dog_pct"] = dogs
future_df

<ipython-input-12-a2d709097410>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_pct"] = favs
<ipython-input-12-a2d709097410>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_pct"] = dogs


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,prediction,fav_pct,dog_pct
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,0,0,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148.0,606.0,382.0,1659.0,845.0,

In [13]:
future_df["fav_pct"] = future_df["fav_pct"].round(2)
future_df["dog_pct"] = future_df["dog_pct"].round(2)
future_df

<ipython-input-13-dd52a5942276>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_pct"] = future_df["fav_pct"].round(2)
<ipython-input-13-dd52a5942276>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_pct"] = future_df["dog_pct"].round(2)


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,prediction,fav_pct,dog_pct
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,0,0,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148.0,606.0,382.0,1659.0,845.0,

In [14]:
future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)
future_df

<ipython-input-14-e94fd81e017f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["fav_moneyline"] = (future_df["fav_pct"] * 100) / (100 - (future_df["fav_pct"] * 100)) * (-100)
<ipython-input-14-e94fd81e017f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  future_df["dog_moneyline"] = ((100 - (future_df["dog_pct"] * 100)) / (future_df["dog_pct"] * 100) * 100)


,lev1,blue_age,red_age,blue_heightIN,red_heightIN,blue_name,red_name,blue_reach,red_reach,blue_stance,red_stance,blue_weight,red_weight,blue_cumsum_tot_stks_lnd,red_cumsum_tot_stks_lnd,blue_cumsum_tot_stks_att,red_cumsum_tot_stks_att,blue_cumsum_sig_stks_lnd,red_cumsum_sig_stks_lnd,blue_cumsum_sig_stks_att,red_cumsum_sig_stks_att,blue_cumsum_TDs_lnd,red_cumsum_TDs_lnd,blue_cumsum_TDs_att,red_cumsum_TDs_att,blue_cumsum_sub_att,red_cumsum_sub_att,blue_cumsum_ctrl_sec,red_cumsum_ctrl_sec,blue_cumsum_KDs,red_cumsum_KDs,blue_cumsum_revs,red_cumsum_revs,blue_cumsum_sig_body_lnd,red_cumsum_sig_body_lnd,blue_cumsum_sig_body_att,red_cumsum_sig_body_att,blue_cumsum_sig_head_lnd,red_cumsum_sig_head_lnd,blue_cumsum_sig_head_att,red_cumsum_sig_head_att,blue_cumsum_sig_leg_lnd,red_cumsum_sig_leg_lnd,blue_cumsum_sig_leg_att,red_cumsum_sig_leg_att,blue_cumsum_sig_dist_lnd,red_cumsum_sig_dist_lnd,blue_cumsum_sig_dist_att,red_cumsum_sig_dist_att,blue_cumsum_sig_clch_lnd,red_cumsum_sig_clch_lnd,blue_cumsum_sig_clch_att,red_cumsum_sig_clch_att,blue_cumsum_sig_gnd_lnd,red_cumsum_sig_gnd_lnd,blue_cumsum_sig_gnd_att,red_cumsum_sig_gnd_att,blue_cumsum_tot_stks_abs,red_cumsum_tot_stks_abs,blue_cumsum_tot_stks_seen,red_cumsum_tot_stks_seen,blue_cumsum_sig_stks_abs,red_cumsum_sig_stks_abs,blue_cumsum_sig_stks_seen,red_cumsum_sig_stks_seen,blue_cumsum_TDs_abs,red_cumsum_TDs_abs,blue_cumsum_TDs_seen,red_cumsum_TDs_seen,blue_cumsum_sub_esc,red_cumsum_sub_esc,blue_cumsum_ctrld_sec,red_cumsum_ctrld_sec,blue_cumsum_KedD,red_cumsum_KedD,blue_cumsum_reversed,red_cumsum_reversed,blue_cumsum_sig_body_abs,red_cumsum_sig_body_abs,blue_cumsum_sig_body_seen,red_cumsum_sig_body_seen,blue_cumsum_sig_head_abs,red_cumsum_sig_head_abs,blue_cumsum_sig_head_seen,red_cumsum_sig_head_seen,blue_cumsum_sig_leg_abs,red_cumsum_sig_leg_abs,blue_cumsum_sig_leg_seen,red_cumsum_sig_leg_seen,blue_cumsum_sig_dist_abs,red_cumsum_sig_dist_abs,blue_cumsum_sig_dist_seen,red_cumsum_sig_dist_seen,blue_cumsum_sig_clch_abs,red_cumsum_sig_clch_abs,blue_cumsum_sig_clch_seen,red_cumsum_sig_clch_seen,blue_cumsum_sig_gnd_abs,red_cumsum_sig_gnd_abs,blue_cumsum_sig_gnd_seen,red_cumsum_sig_gnd_seen,blue_cumsum_fight_sec,red_cumsum_fight_sec,blue_cumsum_decWin,red_cumsum_decWin,blue_cumsum_subWin,red_cumsum_subWin,blue_cumsum_KOWin,red_cumsum_KOWin,blue_cumsum_Decloss,red_cumsum_Decloss,blue_cumsum_subLoss,red_cumsum_subLoss,blue_cumsum_KOLoss,red_cumsum_KOLoss,blue_wins,red_wins,blue_losses,red_losses,blue_titleFights,red_titleFights,red_total_stk_per_min,red_sig_stk_per_min,red_total_stk_acc,red_sig_stk_acc,red_total_stk_abs_per_min,red_sig_stk_abs_per_min,red_stk_head_per_min,red_stk_body_per_min,red_stk_leg_per_min,red_stk_dist_per_min,red_stk_clch_per_min,red_stk_gnd_per_min,red_stk_head_abs_per_min,red_stk_body_abs_per_min,red_stk_leg_abs_per_min,red_stk_dist_abs_per_min,red_stk_clch_abs_per_min,red_stk_gnd_abs_per_min,red_total_stk_per_min_diff,red_sig_stk_per_min_diff,red_ctrl_sec_per_min,red_ctrld_sec_per_min,red_ctrl_diff,red_TD_acc,red_TD_def,red_KDs_per_100_stk,red_reach_dif,red_height_dif,blue_total_stk_per_min,blue_sig_stk_per_min,blue_total_stk_acc,blue_sig_stk_acc,blue_total_stk_abs_per_min,blue_sig_stk_abs_per_min,blue_total_stk_per_min_diff,blue_sig_stk_per_min_diff,blue_ctrl_sec_per_min,blue_ctrld_sec_per_min,blue_ctrl_diff,blue_TD_acc,blue_TD_def,blue_KDs_per_100_stk,blue_stk_head_per_min,blue_stk_body_per_min,blue_stk_leg_per_min,blue_stk_dist_per_min,blue_stk_clch_per_min,blue_stk_gnd_per_min,blue_stk_head_abs_per_min,blue_stk_body_abs_per_min,blue_stk_leg_abs_per_min,blue_stk_dist_abs_per_min,blue_stk_clch_abs_per_min,blue_stk_gnd_abs_per_min,blue_reach_dif,blue_height_dif,winner,prediction,fav_pct,dog_pct,fav_moneyline,dog_moneyline
0,0,30.0,37.0,73.0,72.0,Sean Strickland,Uriah Hall,76.0,79.0,0,0,185,185,913.0,628.0,2176.0,1138.0,808.0,521.0,2052.0,1021.0,12.0,7.0,20.0,18.0,4.0,2.0,1086.0,710.0,4.0,8.0,0.0,4.0,154.0,90.0,277.0,140.0,395.0,317.0,1332.0,781.0,140.0,130.0,189.0,148

In [15]:
future_df["fav_moneyline"] = future_df["fav_moneyline"].astype(int)
future_df["dog_moneyline"] = future_df["dog_moneyline"].astype(int)
future_df

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
future_df.to_csv("data/neaural_future_preds.csv")

In [ ]:
my_picks = future_df[["blue_name", "red_name", "prediction", "fav_pct", "dog_pct", "fav_moneyline", "dog_moneyline"]]
my_picks.to_csv("data/neaural_picks.csv")
my_picks